# Loading libraries and data

In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df_train = pd.read_csv('event-log-training.csv')

# Convert timestamp to datetime

In [4]:
date_list = []

for time in df_train['event time:timestamp']:
    datex = time[:-4]
    date = datetime.strptime(datex, '%d-%m-%Y %H:%M:%S')

    date_list.append(date)

In [5]:
df_train['time and date'] = date_list

In [6]:
df_train

,eventID,case concept:name,case description,case Class,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,time and date
0,0,1,Simulated process instance,Print,PRN1,Job,start,01-01-1970 01:00:00.000,1970-01-01 01:00:00
1,1,1,Simulated process instance,Print,PRN1,Remote Print,complete,01-01-1970 01:15:00.000,1970-01-01 01:15:00
2,2,1,Simulated process instance,Print,PRN1,Read Print Options,complete,01-01-1970 01:26:00.000,1970-01-01 01:26:00
3,3,1,Simulated process instance,Print,PRN1,Rasterization,start,01-01-1970 01:38:00.000,1970-01-01 01:38:00
4,4,1,Simulated process instance,Print,PRN1,Interpretation,start,01-01-1970 01:51:00.000,1970-01-01 01:51:00
...,...,...,...,...,...,...,...,...,...
36623,42949673754,18,Simulated process instance,Print,PRN1,Pressure Roller Spin Stop,complete,25-12-1970 15:51:00.000,1970-12-25 15:51:00
36624,42949673755,18,Simulated process instance,Print,PRN1,Fusing,complete,25-12-1970 16:05:00.000,1970-12-25 16:05:00
36625,42949673756,18,Simulated process instance,Print,PRN1,Wipe Toner on Drum,complete,25-12-1970 16:10:00.000,1970-12-25 16:10:00
36626,42949673757,18,Simulated process instance,Print,PRN1,Erase Charge on Drum,complete,25-12-1970 16:22:00.000,1970-12-25 16:22:00


# Next event and time, and predicted next event and time

In [7]:
cases = list(df_train['case concept:name'].unique())

In [8]:
df_result = pd.DataFrame()

In [9]:
int_dict = {}
for case in cases:
    df_case = df_train[df_train['case concept:name'] == case].reset_index()
    for index, row in df_case.iterrows():
        if row['event concept:name'] in int_dict:
            if index in int_dict[row['event concept:name']]:
                int_dict[row['event concept:name']][index] += 1
            else:
                int_dict[row['event concept:name']].update({index: 1})
        else:
            int_dict[row['event concept:name']] = {index: 1}

In [10]:
#2140
position_dict = {}

for i in range(2140):
    init = 0
    task = 0
    for key in int_dict.keys():
        try:
            new = int_dict[key][i]
        except:
            new = 0
        if new > init:
            init = new
            task = key

    position_dict.update({i: task})

In [11]:
position_dict

{0: 'Job',
 1: 'Remote Print',
 2: 'Read Print Options',
 3: 'Rasterization',
 4: 'Interpretation',
 5: 'Move Scan Head',
 6: 'Interpretation',
 7: 'A/D Conversion',
 8: 'Interpolation',
 9: 'Filtered Image',
 10: 'Collect Image',
 11: 'Illuminate Document',
 12: 'Move Scan Head',
 13: 'Focus Light Beam',
 14: 'A/D Conversion',
 15: 'Interpolation',
 16: 'Filtered Image',
 17: 'Illuminate Document',
 18: 'Accumulate Images',
 19: 'Move Scan Head',
 20: 'Focus Light Beam',
 21: 'A/D Conversion',
 22: 'Interpolation',
 23: 'Filtered Image',
 24: 'Collect Image',
 25: 'Illuminate Document',
 26: 'Move Scan Head',
 27: 'Focus Light Beam',
 28: 'A/D Conversion',
 29: 'Interpolation',
 30: 'Filtered Image',
 31: 'Illuminate Document',
 32: 'Collect Image',
 33: 'Move Scan Head',
 34: 'Focus Light Beam',
 35: 'A/D Conversion',
 36: 'Interpolation',
 37: 'Filtered Image',
 38: 'Illuminate Document',
 39: 'Collect Image',
 40: 'Move Scan Head',
 41: 'Focus Light Beam',
 42: 'A/D Conversion',
 4

In [12]:
# #create next time list
#     nexttime_lst1 = [time for time in df_case['time and date']]
#     nexttime_lst = nexttime_lst1[1:]
#     nexttime_lst.append(nexttime_lst[-1])
    
#     #create difference list
#     time_diff = []
#     for i in range(len(nexttime_lst)):
#         time_diff.append(nexttime_lst[i] - nexttime_lst1[i])

In [13]:
df_result = pd.DataFrame()

for case in cases: 
    
    df_case = df_train[df_train['case concept:name'] == case].copy().reset_index()

    pred_list = [position_dict[i] for i in range(len(df_case))]
    pred_list = pred_list[1:]
    pred_list.append('-')

    

    
    #create next event list
    event_lst = [event for event in df_case['event concept:name']]
    event_lst = event_lst[1:]
    event_lst.append('-')
    
    #create next time list
    nexttime_lst1 = [time for time in df_case['time and date']]
    nexttime_lst = nexttime_lst1[1:]
    nexttime_lst.append(nexttime_lst[-1])
    
    #create difference list
    time_diff = []
    for i in range(len(nexttime_lst)):
        time_diff.append(nexttime_lst[i] - nexttime_lst1[i])
    
    #append columns to df case
    df_case['Next event'] = event_lst
    df_case['Time to next event'] = time_diff


    df_case['Event prediction'] = pred_list
    
    #append df case to final df
    df_result = df_result.append(df_case)

In [14]:
df_result.head()

,index,eventID,case concept:name,case description,case Class,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,time and date,Next event,Time to next event,Event prediction
0,0,0,1,Simulated process instance,Print,PRN1,Job,start,01-01-1970 01:00:00.000,1970-01-01 01:00:00,Remote Print,0 days 00:15:00,Remote Print
1,1,1,1,Simulated process instance,Print,PRN1,Remote Print,complete,01-01-1970 01:15:00.000,1970-01-01 01:15:00,Read Print Options,0 days 00:11:00,Read Print Options
2,2,2,1,Simulated process instance,Print,PRN1,Read Print Options,complete,01-01-1970 01:26:00.000,1970-01-01 01:26:00,Rasterization,0 days 00:12:00,Rasterization
3,3,3,1,Simulated process instance,Print,PRN1,Rasterization,start,01-01-1970 01:38:00.000,1970-01-01 01:38:00,Interpretation,0 days 00:13:00,Interpretation
4,4,4,1,Simulated process instance,Print,PRN1,Interpretation,start,01-01-1970 01:51:00.000,1970-01-01 01:51:00,Unformatted Text,0 days 00:18:00,Move Scan Head


In [15]:
time_dict = {}
for case in cases:
    df_case = df_result[df_result['case concept:name'] == case].copy()
    df_case.reset_index(inplace=True)
    for index, row in df_case.iterrows():
        if index in time_dict:
            time_dict[index]['sum'] += row['Time to next event']
            time_dict[index]['count'] += 1
        else:
            time_dict[index] = {'sum': row['Time to next event'], 'count': 1}
    

    

In [16]:
mean_time_dict = {}
for position in time_dict.keys():
    mean_time = time_dict[position]['sum'] / time_dict[position]['count']
    mean_time_dict[position] = mean_time

In [17]:
mean_time_dict

{0: Timedelta('164 days 09:23:36.750000'),
 1: Timedelta('0 days 00:14:24'),
 2: Timedelta('0 days 00:15:29.250000'),
 3: Timedelta('0 days 00:15:31.500000'),
 4: Timedelta('0 days 00:15:10.500000'),
 5: Timedelta('0 days 00:15:08.250000'),
 6: Timedelta('0 days 00:14:06.750000'),
 7: Timedelta('0 days 00:10:15.750000'),
 8: Timedelta('0 days 00:10:15'),
 9: Timedelta('0 days 00:11:44.250000'),
 10: Timedelta('0 days 00:04:41.250000'),
 11: Timedelta('0 days 00:12:03'),
 12: Timedelta('0 days 00:11:21.750000'),
 13: Timedelta('0 days 00:10:21'),
 14: Timedelta('0 days 00:10:30.750000'),
 15: Timedelta('0 days 00:10:29.250000'),
 16: Timedelta('0 days 00:09:24'),
 17: Timedelta('0 days 00:04:30.750000'),
 18: Timedelta('0 days 00:11:17.250000'),
 19: Timedelta('0 days 00:10:30'),
 20: Timedelta('0 days 00:11:03'),
 21: Timedelta('0 days 00:12:23.250000'),
 22: Timedelta('0 days 00:11:48'),
 23: Timedelta('0 days 00:11:29.250000'),
 24: Timedelta('0 days 00:03:54'),
 25: Timedelta('0 day

In [18]:
lst = []
for case in cases:
    df_case = df_result[df_result['case concept:name'] == case].copy()
    df_case.reset_index(inplace=True)
    for index, row in df_case.iterrows():
        time = mean_time_dict[index]
        lst.append(time)

        



In [19]:
df_result['Time prediction'] = lst

In [20]:
int_dict = {}
for case in cases:
    df_case = df_train[df_train['case concept:name'] == case].reset_index()
    for index, row in df_case.iterrows():
        if row['event concept:name'] in int_dict:
            if index in int_dict[row['event concept:name']]:
                int_dict[row['event concept:name']][index] += 1
            else:
                int_dict[row['event concept:name']].update({index: 1})
        else:
            int_dict[row['event concept:name']] = {index: 1}

In [21]:
df_result.head()

,index,eventID,case concept:name,case description,case Class,event org:resource,event concept:name,event lifecycle:transition,event time:timestamp,time and date,Next event,Time to next event,Event prediction,Time prediction
0,0,0,1,Simulated process instance,Print,PRN1,Job,start,01-01-1970 01:00:00.000,1970-01-01 01:00:00,Remote Print,0 days 00:15:00,Remote Print,164 days 09:23:36.750000
1,1,1,1,Simulated process instance,Print,PRN1,Remote Print,complete,01-01-1970 01:15:00.000,1970-01-01 01:15:00,Read Print Options,0 days 00:11:00,Read Print Options,0 days 00:14:24
2,2,2,1,Simulated process instance,Print,PRN1,Read Print Options,complete,01-01-1970 01:26:00.000,1970-01-01 01:26:00,Rasterization,0 days 00:12:00,Rasterization,0 days 00:15:29.250000
3,3,3,1,Simulated process instance,Print,PRN1,Rasterization,start,01-01-1970 01:38:00.000,1970-01-01 01:38:00,Interpretation,0 days 00:13:00,Interpretation,0 days 00:15:31.500000
4,4,4,1,Simulated process instance,Print,PRN1,Interpretation,start,01-01-1970 01:51:00.000,1970-01-01 01:51:00,Unformatted Text,0 days 00:18:00,Move Scan Head,0 days 00:15:10.500000


# Accuracy

In [22]:
correct_event = 0 
correct_time = 0
total = 0
for index, row in df_result.iterrows():
    total += 1
    if row['Next event'] == row['Event prediction']:
        correct_event += 1
    if row['Time to next event'] == row['Time prediction']:
        correct_time += 1
        
accuracy_event = correct_event/total 
accuracy_time = correct_time/total

accuracy_event, accuracy_time

(0.2314349677842088, 0.016790433548105273)